In [1]:
import os
import torch
import glob
from torchvision import models, transforms
from PIL import Image, ImageDraw
import torch.nn as nn
from annoy import AnnoyIndex

In [2]:
images = glob.glob('../database/*.jpg')
len(images)

689

In [3]:
weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
model = models.mobilenet_v3_large(weights=weights)
model.classifier = nn.Identity()
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [4]:
transform = transforms.Compose([
    transforms.Resize((640,640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
annoy_index = AnnoyIndex(960, 'angular')

In [6]:
for i in range(len(images)):
    # image_path = os.path.join(image_folder, images[i])
    image = Image.open(images[i])
    # print(image)
    image_tensor = transform(image).unsqueeze(0)
    output_tensor = model(image_tensor)
    annoy_index.add_item(i, output_tensor[0])

#     if i%100 == 0:
#         print(f'{i}th image proceeds')

annoy_index.build(10)
annoy_index.save('productDbIndex_MobileNet_V3_Large_Weights.ann')

True